In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision

In [50]:
class Branch(nn.Module):
    
    def __init__(self, in_channel, out_channel, dilation):
        super().__init__()
        self.conv6 = nn.Conv2d(in_channel, 256, kernel_size=3, stride=1, padding=dilation, dilation=dilation, bias=True)
        self.bn6 = nn.BatchNorm2d(256)
        self.conv7 = nn.Conv2d(256, 64, kernel_size=1, stride=1, padding=0, bias=True)
        self.bn7 = nn.BatchNorm2d(64) 
        self.conv8 = nn.Conv2d(64, out_channel, kernel_size=1, stride=1, padding=0, bias=True)
        self.bn8 = nn.BatchNorm2d(out_channel)
    
    def forward(self, x):
        x = F.relu(self.bn6(self.conv6(x)))
        x = F.relu(self.bn7(self.conv7(x)))
        return F.relu(self.bn8(self.conv8(x)))

In [51]:
class ASPP(nn.Module):
    
    def __init__(self, in_channel, out_channel, pyramids):
        super().__init__()
        self.branch1 = Branch(in_channel, out_channel, pyramids[0])
        self.branch2 = Branch(in_channel, out_channel, pyramids[1])
        self.branch3 = Branch(in_channel, out_channel, pyramids[2])
        self.branch4 = Branch(in_channel, out_channel, pyramids[3])
        
        for m in self.children():
            if isninstance(m, nn.Conv2d):
                
        
    def forward(self, x):
        h = 0.0
        h += self.branch1(x)
        h += self.branch2(x)
        h += self.branch3(x)
        h += self.branch4(x)
        return h

In [59]:
class DeepLabV1(nn.Module):
    def __init__(self, n_classes):
        super().__init__()
        vgg = torchvision.models.vgg16_bn(pretrained=True)
        self.vgg = vgg.features
        self.vgg[-1] = nn.MaxPool2d(kernel_size=2, stride=1, padding=0, dilation=1, ceil_mode=False)
        
        self.aspp = ASPP(512, n_classes, [6, 12, 18, 24])
    
    def forward(self, x):
        x = self.vgg(x)
        x = self.aspp(x)
        return x
    
    def freeze_bn(self):
        for m in self.vgg.children():
            if isinstance(m, nn.BatchNorm2d):
                m.eval()